## SQL ##

*En esta fase del proceso de ETL  vamos a crear paso a paso una base de datos en sql desde python, para despúes crear las tablas con sus PK y sus FK.*

*Una vez todo esto este creado procedemos a poblarla desde python con nuestro df limpio y actualizado.*

Este es el codigo usado en sql para crear nuestra base de datos. *Este codigo se puede usar para crear la base de datos directamente desde SQL,pero vamos a crearla desde este entorno*

In [ ]:
"""
CREATE DATABASE IF NOT EXIST AIRBB
USE AIRBB

CREATE A TABLE Hosting (
record_id BIGINT UNSIGNED,
Titles TEXT,
Property_types TEXT,
Host_name TEXT,
PRIMARY KEY (Ids),
);

CREATE A TABLE Description_ (
id BIGINT UNSIGNED,
Host_name TEXT,
Prices_per_night INT,
Check_ins TEXT,
Check_outs TEXT,
Cleaning_fees INT,
Maximum_guests INT,
Location TEXT,
Beds INT,
Baths INT,
Rooms INT,
FOREING KEY (id) REFERENCE Hosting (record_id)

);

CREATE A TABLE Ratings (
record_id BIGINT UNSIGNED,
Ratings INT,
Num_reviews INT,
FOREING KEY (record_id) REFERENCE Hosting (record_id)
);
"""

In [1]:
import pandas as pd
import mysql.connector
import numpy as np

Este es el codigo para crear desde python la base de datos en sql.

*Correr solo una vez*

In [ ]:
db = mysql.connector.connect(
     host="localhost",
     user="root",
     password="contraseña123"#Aqui teneis que poner vuestra contraeña.
)

cursor = db.cursor()

# Crear la base de datos si no existe
cursor.execute('CREATE DATABASE IF NOT EXISTS AIRBB')

# Conectar a la base de datos recién creada
db.database = 'AIRBB'

# Crear la tabla Hosting
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Hosting (
        record_id BIGINT UNSIGNED PRIMARY KEY,
        Titles TEXT,
        Property_types TEXT,
        Host_name TEXT
    );
''')

# Crear la tabla Description_
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Description_ (
        id BIGINT UNSIGNED,
        Host_name TEXT,
        Prices_per_night INT,
        Check_ins TEXT,
        Check_outs TEX,
        Cleaning_fees INT,
        Maximum_guests INT,
        Location TEXT,
        Camas INT,
        Baños INT,
        Dormitorios INT,
        FOREIGN KEY (id) REFERENCES Hosting(record_id)
    );
''')

# Crear la tabla Ratings
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Ratings (
        record_id BIGINT UNSIGNED,
        Ratings INT,
        Num_reviews INT,
        FOREIGN KEY (record_id) REFERENCES Hosting(record_id)
    );
''')

# Cerrar el cursor y la conexión
cursor.close()
db.close()


Este es el codigo para la subida de nuestro df a la base de datos

In [2]:
df = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_clean.csv')


Dividimos las columnas del df en distintos df segun las tablas de nuestra base de datos.

*Elinamos las filas que tengan duplicada la columna record_id*

In [3]:
df_sin_duplicados = df.drop_duplicates(subset='record_id', keep=False)

In [4]:
df_Hosting = df_sin_duplicados[['record_id','Titles','Property_types','Host_name']]

In [5]:
df_Hosting.head()

,record_id,Titles,Property_types,Host_name
2,1268163818980729724,¡Bienvenido a tu escapada perfecta en el coraz...,Alojamiento entero,Joonas
3,44247318,ACOGEDOR APARTAMENTO DE PLAYA DE 1 DORMITORIO ...,Alojamiento entero,Jan
5,545602057442226577,Acogedora habitación privada en el centro de B...,Habitación,Magdalena
10,1093382940108512105,"En una de las calles principales de la zona, a...",Habitación,Guido
12,549976913937241380,Disfruta de la sencillez de este alojamiento t...,Habitación,Danny


In [6]:
df_Description_ = df_sin_duplicados[['record_id','Host_name', 'Prices_per_night','Check_ins','Check_outs','Cleaning_fees','Maximum_guests','Location','Camas','Baños','Dormitorios']]
df_Description_ = df_Description_.rename(columns={'record_id': 'id'})


In [7]:
df_Ratings = df_sin_duplicados[['record_id','Ratings','Num_reviews']]

Aqui nos aseguramos de que no haya ningun nan sustituyendolos todos por none (valor que si admite sql).

In [8]:
df_Description_ = df_Description_.replace({np.nan : None}) #Sustituimos todos los posibles nan por none para poder subirlos
df_Hosting = df_Hosting.replace({np.nan : None})
df_Ratings = df_Ratings.replace({np.nan : None})

Aqui procedemos a la subida de las distintas tablas.

Hosting

In [ ]:
column_names= ['record_id','Titles','Property_types','Host_name']

In [ ]:
database = "AIRBB"
table_name = "Hosting"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_Hosting.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_Hosting[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Description_

In [10]:
column_names = ['id','Host_name', 'Prices_per_night','Check_ins','Check_outs','Cleaning_fees','Maximum_guests','Location','Camas','Baños','Dormitorios']

In [11]:
database = "AIRBB"
table_name = "Description_"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_Description_.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_Description_[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 384 filas


Ratings

No consigo que funcione el resto esta tdo okey 

In [9]:
column_names= ['record_id','Ratings','Num_reviews']

In [10]:
database = "AIRBB"
table_name = "Ratings"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_Ratings.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_Ratings[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`airbb`.`ratings`, CONSTRAINT `record_id` FOREIGN KEY (`record_id`) REFERENCES `hosting` (`record_id`))

# Extracción de datos

In [11]:
def extraer_datos_mysql(host, user, password, database, query):
    # Conectarse a la base de datos
    db = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Crear cursor para ejecutar la consulta
    cursor = db.cursor()

    # Ejecutar la consulta
    cursor.execute(query)
    
    # Obtener los nombres de las columnas
    column_names = cursor.column_names
    print("Nombres de las columnas:", column_names)

    # Guardar y mostrar los datos de la tabla
    data = cursor.fetchall()
    for fila in data:
        print(fila)

    # Cerrar el cursor y la conexión
    cursor.close()
    db.close()


Hosting

In [12]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBB"
query = "SELECT * FROM Hosting;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('record_id', 'Titles', 'Property_types', 'Host_name')
(40983, '¡Nuestro fresco y elegante apartamento loft clásico con concepto abierto y abierto tiene un verdadero toque urbano! Tiene capacidad para 4 personas con 2 en la cama de matrimonio y 2 en el sofá cama, y está situado en el centro, a solo 5 manzanas de La Pedrera. No tenemos ascensor, pero estamos en la primera planta y justo debajo de una brasería local típica. ¡Disfruta de un fácil acceso a las principales líneas de metro y autobús!El espacioComo nos dicen nuestros huéspedes, es perfecto para pasar unos días de visita a Barcelona. ¡Camina desde todo lo que necesitas! estación de metro y autobús, Sagrada Familia, La Pedrera, Las Ramblas y el centro de la ciudad.Nuestro Loft Classical Apartment, con vigas de madera expuestas y restauradas con construcción de más de 100 años de antigüedad, tiene el encanto de una construcción tradicional típica de Barcelona, pero también su particularidad por no ser un

Description_

In [14]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBB"
query = "SELECT * FROM Description_;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('id', 'Host_name', 'Prices_per_night', 'Check_ins', 'Check_outs', 'Cleaning_fees', 'Maximum_guests', 'Location', 'Camas', 'Baños', 'Dormitorios')
(1268163818980729724, 'Joonas', 147, 'Llegada a partir de las 15:00', 'Salida antes de las 12:00', 126, 4, 'Barcelona, Catalunya, España', 2, 1, 2)
(44247318, 'Jan', 55, 'Horario de llegada: de 18:00 a 23:00', 'Salida antes de las 11:00', 80, 2, 'Barcelona, Catalunya, España', 1, 1, 1)
(545602057442226577, 'Magdalena', 50, 'Horario de llegada: de 15:00 a 0:00', 'Salida antes de las 12:00', 43, 1, 'Barcelona, Catalunya, España', 1, None, None)
(1093382940108512105, 'Guido', 59, 'Horario de llegada: de 13:00 a 15:00', 'Salida antes de las 10:00', 50, 2, 'Barcelona, Catalunya, España', 1, None, None)
(549976913937241380, 'Danny', 50, 'Llegada a partir de las 15:00', 'Salida antes de las 11:00', 25, 3, 'Barcelona, Catalunya, España', 1, None, None)
(1212272386691330030, 'Mada', 95, 'Máximo 4\xa0huéspedes', 'No hay alarma

Ratings

In [15]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBB"
query = "SELECT * FROM Ratings;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('record_id', 'Ratings', 'Num_reviews')
